Importar librerias requeridas

In [12]:
import ee 
import geemap
import json
import os

Autenticación con GEE y extracción de zona de estudio

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-rimartinezs')

Creación del GIF

In [ ]:
geometry = ee.Geometry.Rectangle([-72.25, 11.35, -72.20, 11.40])
# Filtrar la colección de Sentinel-2
collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate('2020-01-01', '2020-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(lambda image: image.clip(geometry)))


In [ ]:

def create_monthly_composites(start_year, end_year):
    composites = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            filtered = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                        .filterBounds(geometry)
                        .filterDate(start_date, end_date)
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))
            composite = filtered.median().clip(geometry)
            composites.append(composite)
    return ee.ImageCollection(composites)

# Crear la colección de compuestos mensuales para 2020
monthly_collection = create_monthly_composites(2020, 2020)


# Visualizar las imágenes en color natural
def visualize_image(image):
    return image.visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000)

rgb_collection = monthly_collection.map(visualize_image)

# Parámetros para la animación
video_args = {
    'dimensions': 720,
    'region': geometry,
    'framesPerSecond': 2,
    'crs': 'EPSG:4326'
}


output_path = os.path.join(os.getcwd(), '..', 'Images', 'Maicao.gif')

# Descargar la animación como GIF
geemap.download_ee_video(rgb_collection, video_args, output_path)




Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\ricar\OneDrive\Maestría en Geomática\Percepción Remota\GitHub\GEE-Remote-Sensing\Images\Maicao.gif
El archivo GIF se ha guardado en: c:\Users\ricar\OneDrive\Maestría en Geomática\Percepción Remota\GitHub\GEE-Remote-Sensing\Notebooks\..\Images\Maicao.gif


Output()